In this project I assumed that I decided to open an Italian restaurant.  
To decide on the neighborhood for the restaurant, I need to find the locations    
of all the restaurants from all neighborhoods using foursquare, and then  
cluster the neighborhoods according to the popularity of the present restaurant types.  
Let's see the neighborhood alternatives for my restaurant!

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Read table
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = tables[0]

#Remove all not assigned labels
df = df[df['Borough'] != 'Not assigned']

#Combine multiple entires
df = df.groupby(['Postal Code','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [3]:
#Read latitude and logtitude from excel file
location_df = pd.read_csv('Geospatial_Coordinates.csv')
location_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [4]:
df = df.sort_values(by = 'Postal Code')
location_df = location_df.sort_values(by = 'Postal Code')
df = df.merge(location_df)
df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


Let's find the latitude and longitude of Toronto

In [5]:
from geopy.geocoders import Nominatim
import folium

address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Draw the map of neighborhoods of Toronto

In [6]:
# create map of Toronto
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#003300',
        fill=True,
        fill_color='#009933',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

Using foursquare credentials we need to take nearby Venues

In [7]:
CLIENT_ID = 'ED50LRDOI5KI2IFP54RPTUJHNPMVF0ZHJV5RGKCGV0T1KXUG' # your Foursquare ID
CLIENT_SECRET = 'HMHZURR30PUCHCJZKWLDAJ1TSD25VGQJTWQDCALGS2MV3MI1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ED50LRDOI5KI2IFP54RPTUJHNPMVF0ZHJV5RGKCGV0T1KXUG
CLIENT_SECRET:HMHZURR30PUCHCJZKWLDAJ1TSD25VGQJTWQDCALGS2MV3MI1


In [8]:
import requests

#For each neighborhood, query for all nearby venues 
def getNearbyVenues(names, latitudes, longitudes, radius= 1000, LIMIT = 1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'])


Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence P

In [10]:
#List of all toronto venues
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Harvey's,43.800020,-79.198307,Restaurant
1,"Malvern, Rouge",43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant
2,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
3,"Malvern, Rouge",43.806686,-79.194353,RBC Royal Bank,43.798782,-79.197090,Bank
4,"Malvern, Rouge",43.806686,-79.194353,Caribbean Wave,43.798558,-79.195777,Caribbean Restaurant
...,...,...,...,...,...,...,...
4910,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437,Panorama Park,43.747021,-79.583497,Park
4911,Northwest,43.706748,-79.594054,Tim Hortons,43.714657,-79.593716,Coffee Shop
4912,Northwest,43.706748,-79.594054,Saand Rexdale,43.705072,-79.598725,Drugstore
4913,Northwest,43.706748,-79.594054,Woodbine Hotel & Suites,43.712624,-79.592664,Hotel


We only need the rows having Restaurant in venue category

In [11]:
#Select only restaurants
selected_samples = []
for i in range(toronto_venues.shape[0]):
    if 'Restaurant' in toronto_venues['Venue Category'].values[i]:
        selected_samples.append(i)
toronto_venues = toronto_venues.iloc[selected_samples]
toronto_venues


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Harvey's,43.800020,-79.198307,Restaurant
1,"Malvern, Rouge",43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant
2,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
4,"Malvern, Rouge",43.806686,-79.194353,Caribbean Wave,43.798558,-79.195777,Caribbean Restaurant
11,"Malvern, Rouge",43.806686,-79.194353,Mr Jerk,43.801262,-79.199758,African Restaurant
...,...,...,...,...,...,...,...
4861,Westmount,43.696319,-79.532242,Mayflower Chinese Food,43.692753,-79.531566,Chinese Restaurant
4877,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724,Gourmet Express,43.693111,-79.557294,Chinese Restaurant
4889,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724,Martingrove Fish and Chips,43.690537,-79.561878,American Restaurant
4904,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437,McDonald's,43.741757,-79.584230,Fast Food Restaurant


In order to be able to cluster we need to do one hot encoding

In [12]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()


,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,...,Sushi Restaurant,Syrian Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Malvern, Rouge",0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
11,"Malvern, Rouge",0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now let's calculate the average number of restaurants in a neighborhood for each category

In [13]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,...,Sushi Restaurant,Syrian Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.0,0.045455,0.045455,0.0,0.0,0.0,0.045455,0.090909,...,0.045455,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.045455
1,"Alderwood, Long Branch",0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.250000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
3,Bayview Village,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.083333,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.083333,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,Willowdale,0.0,0.0,0.025000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.075000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.025000
86,"Willowdale, Newtonbrook",0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
87,Woburn,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
88,Woodbine Heights,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.000000


Now it's time to cluster the neighborhoods by using kmeans, suppose the number of clusters is 10

In [14]:
#Cluster neighborhoods
from sklearn.cluster import KMeans

# run k-means clustering
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=10, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 5, 1, 7, 1, 7, 9, 7, 1, 1], dtype=int32)

In [15]:
#Combine cluster labels and top restaurant with dataframe
df = df.sort_values(by = 'Neighborhood')
final_df = pd.concat([toronto_grouped['Neighborhood'], pd.DataFrame(kmeans.labels_, columns = ['Cluster Label'])], axis = 1)

top_restaurants = toronto_grouped.columns[1:][np.argmax(toronto_grouped.values[:, 1:], axis = 1)]
final_df = pd.concat([final_df, pd.DataFrame(top_restaurants, columns = ['Most Common Restaurant'])], axis = 1)

final_df = pd.merge(final_df, df, on = 'Neighborhood')
final_df

,Neighborhood,Cluster Label,Most Common Restaurant,Postal Code,Borough,Latitude,Longitude
0,Agincourt,3,Chinese Restaurant,M1S,Scarborough,43.794200,-79.262029
1,"Alderwood, Long Branch",5,Moroccan Restaurant,M8W,Etobicoke,43.602414,-79.543484
2,"Bathurst Manor, Wilson Heights, Downsview North",1,Mediterranean Restaurant,M3H,North York,43.754328,-79.442259
3,Bayview Village,7,Japanese Restaurant,M2K,North York,43.786947,-79.385975
4,"Bedford Park, Lawrence Manor East",1,Italian Restaurant,M5M,North York,43.733283,-79.419750
...,...,...,...,...,...,...,...
90,Willowdale,8,Korean Restaurant,M2N,North York,43.770120,-79.408493
91,"Willowdale, Newtonbrook",8,Korean Restaurant,M2M,North York,43.789053,-79.408493
92,Woburn,0,Chinese Restaurant,M1G,Scarborough,43.770992,-79.216917
93,Woodbine Heights,9,Thai Restaurant,M4C,East York,43.695344,-79.318389


Draw the map of clusters

In [16]:
#from geopy.geocoders import Nominatim
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(10)
ys = [i + x + (i*x)**2 for i in range(10)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(final_df['Latitude'], final_df['Longitude'], final_df['Neighborhood'], final_df['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Let's look at the dataframes of all the clusters one by one

In [17]:
final_df.loc[final_df['Cluster Label'] == 0]

,Neighborhood,Cluster Label,Most Common Restaurant,Postal Code,Borough,Latitude,Longitude
12,Cedarbrae,0,Indian Restaurant,M1H,Scarborough,43.773136,-79.239476
24,"Dorset Park, Wexford Heights, Scarborough Town...",0,Asian Restaurant,M1P,Scarborough,43.757410,-79.273304
43,"India Bazaar, The Beaches West",0,Indian Restaurant,M4L,East Toronto,43.668999,-79.315572
69,"Runnymede, The Junction North",0,Asian Restaurant,M6N,York,43.673185,-79.487262
82,Thorncliffe Park,0,Indian Restaurant,M4H,East York,43.705369,-79.349372
92,Woburn,0,Chinese Restaurant,M1G,Scarborough,43.770992,-79.216917


In [18]:
final_df.loc[final_df['Cluster Label'] == 1]

,Neighborhood,Cluster Label,Most Common Restaurant,Postal Code,Borough,Latitude,Longitude
2,"Bathurst Manor, Wilson Heights, Downsview North",1,Mediterranean Restaurant,M3H,North York,43.754328,-79.442259
4,"Bedford Park, Lawrence Manor East",1,Italian Restaurant,M5M,North York,43.733283,-79.419750
8,Business reply mail Processing Centre,1,Fast Food Restaurant,M7Y,East Toronto,43.662744,-79.321558
9,"CN Tower, King and Spadina, Railway Lands, Har...",1,Sushi Restaurant,M5V,Downtown Toronto,43.628947,-79.394420
11,Canada Post Gateway Processing Centre,1,Chinese Restaurant,M7R,Mississauga,43.636966,-79.615819
13,Central Bay Street,1,Japanese Restaurant,M5G,Downtown Toronto,43.657952,-79.387383
15,Church and Wellesley,1,Japanese Restaurant,M4Y,Downtown Toronto,43.665860,-79.383160
16,"Clarks Corners, Tam O'Shanter, Sullivan",1,Fast Food Restaurant,M1T,Scarborough,43.781638,-79.304302
19,Davisville,1,Italian Restaurant,M4S,Central Toronto,43.704324,-79.388790
20,Davisville North,1,Italian Restaurant,M4P,Central Toronto,43.712751,-79.390197


In [19]:
final_df.loc[final_df['Cluster Label'] == 2]

,Neighborhood,Cluster Label,Most Common Restaurant,Postal Code,Borough,Latitude,Longitude
41,Humber Summit,2,Italian Restaurant,M9L,North York,43.756303,-79.565963
58,"Old Mill South, King's Mill Park, Sunnylea, Hu...",2,Italian Restaurant,M8Y,Etobicoke,43.636258,-79.498509
67,"Rouge Hill, Port Union, Highland Creek",2,Italian Restaurant,M1C,Scarborough,43.784535,-79.160497


In [20]:
final_df.loc[final_df['Cluster Label'] == 3]

,Neighborhood,Cluster Label,Most Common Restaurant,Postal Code,Borough,Latitude,Longitude
0,Agincourt,3,Chinese Restaurant,M1S,Scarborough,43.794200,-79.262029
44,"Kennedy Park, Ionview, East Birchmount Park",3,Chinese Restaurant,M1K,Scarborough,43.727929,-79.262029
46,"Kingsview Village, St. Phillips, Martin Grove ...",3,American Restaurant,M9R,Etobicoke,43.688905,-79.554724
51,"Milliken, Agincourt North, Steeles East, L'Amo...",3,Chinese Restaurant,M1V,Scarborough,43.815252,-79.284577
55,"North Park, Maple Leaf Park, Upwood Park",3,Chinese Restaurant,M6L,North York,43.713756,-79.490074
61,Parkwoods,3,Caribbean Restaurant,M3A,North York,43.753259,-79.329656
74,"Steeles West, L'Amoreaux West",3,Chinese Restaurant,M1W,Scarborough,43.799525,-79.318389
87,Westmount,3,Chinese Restaurant,M9P,Etobicoke,43.696319,-79.532242


In [21]:
final_df.loc[final_df['Cluster Label'] == 4]

,Neighborhood,Cluster Label,Most Common Restaurant,Postal Code,Borough,Latitude,Longitude
17,"Cliffside, Cliffcrest, Scarborough Village West",4,Cajun / Creole Restaurant,M1M,Scarborough,43.716316,-79.239476


In [22]:
final_df.loc[final_df['Cluster Label'] == 5]

,Neighborhood,Cluster Label,Most Common Restaurant,Postal Code,Borough,Latitude,Longitude
1,"Alderwood, Long Branch",5,Moroccan Restaurant,M8W,Etobicoke,43.602414,-79.543484


In [23]:
final_df.loc[final_df['Cluster Label'] == 6]

,Neighborhood,Cluster Label,Most Common Restaurant,Postal Code,Borough,Latitude,Longitude
21,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",6,Fast Food Restaurant,M6M,York,43.691116,-79.476013
35,Glencairn,6,Fast Food Restaurant,M6B,North York,43.709577,-79.445073
36,"Golden Mile, Clairlea, Oakridge",6,Fast Food Restaurant,M1L,Scarborough,43.711112,-79.284577
37,"Guildwood, Morningside, West Hill",6,Fast Food Restaurant,M1E,Scarborough,43.763573,-79.188711
47,"Lawrence Manor, Lawrence Heights",6,Fast Food Restaurant,M6A,North York,43.718518,-79.464763
50,"Malvern, Rouge",6,Fast Food Restaurant,M1B,Scarborough,43.806686,-79.194353
60,"Parkview Hill, Woodbine Gardens",6,Fast Food Restaurant,M4B,East York,43.706397,-79.309937
71,"South Steeles, Silverstone, Humbergate, Jamest...",6,Caribbean Restaurant,M9V,Etobicoke,43.739416,-79.588437


In [24]:
final_df.loc[final_df['Cluster Label'] == 7]

,Neighborhood,Cluster Label,Most Common Restaurant,Postal Code,Borough,Latitude,Longitude
3,Bayview Village,7,Japanese Restaurant,M2K,North York,43.786947,-79.385975
5,Berczy Park,7,Japanese Restaurant,M5E,Downtown Toronto,43.644771,-79.373306
7,"Brockton, Parkdale Village, Exhibition Place",7,Restaurant,M6K,West Toronto,43.636847,-79.428191
10,Caledonia-Fairbanks,7,Falafel Restaurant,M6E,York,43.689026,-79.453512
18,"Commerce Court, Victoria Hotel",7,Japanese Restaurant,M5L,Downtown Toronto,43.648198,-79.379817
22,Don Mills,7,Restaurant,M3B,North York,43.745906,-79.352188
23,Don Mills,7,Restaurant,M3C,North York,43.725900,-79.340923
31,"Fairview, Henry Farm, Oriole",7,Japanese Restaurant,M2J,North York,43.778517,-79.346556
32,"First Canadian Place, Underground city",7,Japanese Restaurant,M5X,Downtown Toronto,43.648429,-79.382280
34,"Garden District, Ryerson",7,Japanese Restaurant,M5B,Downtown Toronto,43.657162,-79.378937


In [25]:
final_df.loc[final_df['Cluster Label'] == 8]

,Neighborhood,Cluster Label,Most Common Restaurant,Postal Code,Borough,Latitude,Longitude
14,Christie,8,Korean Restaurant,M6G,Downtown Toronto,43.669542,-79.422564
40,Hillcrest Village,8,Chinese Restaurant,M2H,North York,43.803762,-79.363452
42,Humewood-Cedarvale,8,Israeli Restaurant,M6C,York,43.693781,-79.428191
88,"Wexford, Maryvale",8,Middle Eastern Restaurant,M1R,Scarborough,43.750072,-79.295849
89,Willowdale,8,Korean Restaurant,M2R,North York,43.782736,-79.442259
90,Willowdale,8,Korean Restaurant,M2N,North York,43.770120,-79.408493
91,"Willowdale, Newtonbrook",8,Korean Restaurant,M2M,North York,43.789053,-79.408493


In [26]:
final_df.loc[final_df['Cluster Label'] == 9]

,Neighborhood,Cluster Label,Most Common Restaurant,Postal Code,Borough,Latitude,Longitude
6,"Birch Cliff, Cliffside West",9,Restaurant,M1N,Scarborough,43.692657,-79.264848
93,Woodbine Heights,9,Thai Restaurant,M4C,East York,43.695344,-79.318389


As we see cluster 2 is ideal for my restaurant

In [27]:
final_df.loc[final_df['Cluster Label'] == 2]['Neighborhood'].values

array(['Humber Summit',
       "Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East",
       'Rouge Hill, Port Union, Highland Creek'], dtype=object)

You see I can open my restaurant at, Humber Summit, Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East",Rouge Hill, Port Union or  
Highland Creek, to have a chance of more customers